In [1]:
# !pip3 install -r requirements.txt

In [2]:
import torch
from torch.utils.data import DataLoader
from plotly import express as px
from collections import Counter

from modules.dataset import *
from modules.utility import *
from modules.trainer import *
from modules.dataloader import *
from modules.models import *

torch.manual_seed(1)

In [3]:
dataset = IntelImageClassificationDataset()
# dataset = LinearProbingDataset(conversion_model=SqueezeNet_1_1(out_features=None)) # Linear Probing

https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [4]:
labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

In [5]:
NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset) # cannot be used with LinearProbingDataset

BoundedIntText(value=0, description='item:', max=14033)

In [6]:
dataloader = DataLoader(dataset.train_dataset, batch_size=128, shuffle=True)
# dataloader = NoDataLoader(dataset.train_dataset, batch_size=128, shuffle=True)

In [ ]:
model = SqueezeNet_1_1()
# model = LinearProbe()

In [8]:
print("model size: {} Byte, parameter count: {}".format(*Evaluator.size(model)))

model size: 2902296 Byte, parameter count: 725574


In [9]:
trainer = Trainer(
    model,
    optimizer=torch.optim.SGD(
        model.parameters(),
        lr=0.001,
        momentum=0.99,
        nesterov=True
        )
    )

In [10]:
trainer.train(dataloader, epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# torch.save(model.state_dict(), f"{model.__class__.__name__}.pt")

In [12]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
target = torch.tensor(tuple(item[1] for item in dataset.test_dataset))
print("Accuracy: ", Evaluator.acc(output, target).item())

NotebookPlotter.plot_confusion_matrix(Evaluator.cm(output, target))

Accuracy:  0.9103198051452637


In [13]:
# !tensorboard --logdir=./log